# 4/13  3. Знакомимся с новыми данными: коронавирус

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

`covid_data` — данные о показателях заболеваемости за каждый день в различных регионах 187 стран. 

In [5]:
covid_data = pd.read_csv('data/covid_data.csv').copy()
display(covid_data.head())


,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


Мы будем работать со следующими столбцами:

`date` — дата наблюдения;  
`province`/`state` — наименование провинции/штата;  
`country` — наименование страны;  
`confirmed` — общее число зафиксированных случаев на указанный день;  
`deaths` — общее число зафиксированных смертей на указанный день;  
`recovered` — общее число выздоровлений на указанный день.  

`country_vaccinations` — данные о процессе вакцинирования людей в 221 стране мира.

In [9]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv').copy()
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
vaccinations_data

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.00,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
...,...,...,...,...,...,...,...,...,...
42790,Zimbabwe,2021-09-01,4270430.0,2615233.0,17.33,1655197.0,10.97,36416.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."
42791,Zimbabwe,2021-09-02,4323735.0,2649505.0,17.56,1674230.0,11.09,39711.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."
42792,Zimbabwe,2021-09-03,4372216.0,2681657.0,17.77,1690559.0,11.20,42317.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."
42793,Zimbabwe,2021-09-04,4400246.0,2698332.0,17.88,1701914.0,11.28,41413.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."


Данная таблица содержит следующие столбцы:

`country` — наименование страны;  
`date` — дата наблюдения;  
`total_vaccinations` — общее число введённых вакцин в стране на указанный день;  
`people_vaccinated` — общее число привитых первым компонентом в стране на указанный день;  
`people_vaccinated_per_hundred` — процент привитых первым компонентом в стране на указанный день (рассчитывается как `people_vaccinated`/`population`*100);  
`people_fully_vaccinated` — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);  
`people_fully_vaccinated_per_hundred` — процент привитых вторым компонентом в стране на указанный день (рассчитывается как `people_fully_vaccinated`/`population`*100);  
`daily_vaccination` — ежедневная вакцинация (число вакцинированных в указанный день);  
`vaccines` — комбинации вакцин, используемые в стране.

ПРЕДОБРАБОТКА ДАННЫХ

В таблице covid_data:

1
Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:

In [10]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()
covid_data

,date,country,confirmed,deaths,recovered
0,01/01/2021,Afghanistan,51526.0,2191.0,41727.0
1,01/01/2021,Albania,58316.0,1181.0,33634.0
2,01/01/2021,Algeria,99897.0,2762.0,67395.0
3,01/01/2021,Andorra,8117.0,84.0,7463.0
4,01/01/2021,Angola,17568.0,405.0,11146.0
...,...,...,...,...,...
86780,12/31/2020,Vietnam,1465.0,35.0,1325.0
86781,12/31/2020,West Bank and Gaza,138004.0,1400.0,117183.0
86782,12/31/2020,Yemen,2099.0,610.0,1394.0
86783,12/31/2020,Zambia,20725.0,388.0,18660.0


2
Преобразуем даты в формат `datetime` с помощью функции `pd.to_datetime`():

In [12]:
covid_data['date'] = pd.to_datetime(covid_data['date'])
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86785 entries, 0 to 86784
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       86785 non-null  datetime64[ns]
 1   country    86785 non-null  object        
 2   confirmed  86785 non-null  float64       
 3   deaths     86785 non-null  float64       
 4   recovered  86785 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 3.3+ MB


3
Создадим признак больных на данный момент (`active`). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:

In [13]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']
covid_data

,date,country,confirmed,deaths,recovered,active
0,2021-01-01,Afghanistan,51526.0,2191.0,41727.0,7608.0
1,2021-01-01,Albania,58316.0,1181.0,33634.0,23501.0
2,2021-01-01,Algeria,99897.0,2762.0,67395.0,29740.0
3,2021-01-01,Andorra,8117.0,84.0,7463.0,570.0
4,2021-01-01,Angola,17568.0,405.0,11146.0,6017.0
...,...,...,...,...,...,...
86780,2020-12-31,Vietnam,1465.0,35.0,1325.0,105.0
86781,2020-12-31,West Bank and Gaza,138004.0,1400.0,117183.0,19421.0
86782,2020-12-31,Yemen,2099.0,610.0,1394.0,95.0
86783,2020-12-31,Zambia,20725.0,388.0,18660.0,1677.0


4
Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода `diff()`:

In [19]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()
covid_data.head()

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
11337,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
11570,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11807,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12051,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12299,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


В таблице vaccinations_data достаточно будет преобразовать столбцы в формат `datetime`:



In [15]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])
vaccinations_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42795 entries, 0 to 42794
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   country                              42795 non-null  object        
 1   date                                 42795 non-null  datetime64[ns]
 2   total_vaccinations                   23457 non-null  float64       
 3   people_vaccinated                    22371 non-null  float64       
 4   people_vaccinated_per_hundred        22371 non-null  float64       
 5   people_fully_vaccinated              19462 non-null  float64       
 6   people_fully_vaccinated_per_hundred  19462 non-null  float64       
 7   daily_vaccinations                   42558 non-null  float64       
 8   vaccines                             42795 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 2.9+ MB


Задание 3.1

За какой период представлены данные в таблице `covid_data`? В качестве ответа введите даты в формате `datetime` (без указания времени).
Введите ответ в следующем формате (без пробелов): `ГГГГ-ММ-ДД-ГГГГ-ММ-ДД`

In [17]:
covid_data['date'].describe()

count                            86785
mean     2020-10-15 20:19:34.931151616
min                2020-01-22 00:00:00
25%                2020-06-25 00:00:00
50%                2020-10-17 00:00:00
75%                2021-02-07 00:00:00
max                2021-05-29 00:00:00
Name: date, dtype: object

Задание 3.2

За какой период представлены данные в таблице `vaccinations_data`? В качестве ответа введите даты в формате `datetime` без указания времени.
Введите ответ в следующем формате (без пробелов): `ГГГГ-ММ-ДД-ГГГГ-ММ-ДД`

In [20]:
vaccinations_data['date'].describe()

count                            42795
mean     2021-05-21 12:12:46.182965248
min                2020-12-02 00:00:00
25%                2021-04-02 00:00:00
50%                2021-05-26 00:00:00
75%                2021-07-15 00:00:00
max                2021-09-06 00:00:00
Name: date, dtype: object

Задание 3.3

С помощью метода `merge()` объедините таблицы `covid_data` и `vaccinations_data` по столбцам `date` и `country`.
Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1. То есть в результирующую таблицу должны попасть все записи из таблицы `covid_data` и из её пересечения с `vaccinations_data`, но не более. Результат объединения занесите в переменную `covid_df`.

Сколько строк и столбцов в таблице `covid_df`?  
Введите ответ в виде двух чисел через дефис (например, 333-33): первое число — количество строк, второе число — количество столбцов.

In [23]:
covid_df = covid_data.merge(vaccinations_data,
                 how='left',
                 on=['date', 'country']                 
                 )
covid_df

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86780,2020-03-12,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86781,2020-03-14,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86782,2020-03-15,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86783,2020-03-16,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В получившейся в задании 3.3 таблице `covid_df` создайте признаки `death_rate` — общий процент смертей среди зафиксированных случаев (летальность) и `recover_rate` — общий процент случаев выздоровления. Данные характеристики рассчитайте как отношение числа смертей (`deaths`) и числа выздоровлений (`recovered`) к числу зафиксированных случаев (`confirmed`) и умножьте результаты на `100%`.

In [ ]:
covid_df['death_rate'] = covid_df['deaths']/covid_df['confirmed']*100
covid_df['recover_rate'] = covid_df['recovered']/covid_df['confirmed']*100
covid_df

Задание 3.4

Какова максимальная летальность в США (`United States`) за весь период? Ответ округлите до второго знака после запятой.

In [36]:
round(covid_df.groupby('country')['death_rate'].max()['United States'], 2)

# Решение SkillFactory:
# round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2)

7.19

Задание 3.5

Чему равен средний процент выздоровевших в России (`Russia`)? Ответ округлите до второго знака после запятой.

In [39]:
round(covid_df.groupby('country')['recover_rate'].mean()['Russia'], 2)


67.06

# 5/13  4. Графические возможности библиотеки Pandas

БАЗОВАЯ ВИЗУАЛИЗАЦИЯ В PANDAS

Основным методом для создания простейших графиков в Pandas является `plot()`.

Основные параметры метода `plot()`

`x` — название признака, который будет отложен по оси абсцисс;  
`y` — название признака, который будет отложен по оси ординат;  
`figsize` — размер графика (кортеж из ширины и высоты в дюймах);  
`kind` —тип визуализации. Основные типы:  
- '`line`' — линейный график (по умолчанию);  
- '`bar`' — столбчатая диаграмма;  
- '`barh`' — горизонтальная столбчатая диаграмма;  
- '`hist`' — гистограмма;  
- '`box`' — коробчатая диаграмма (`boxplot`);  
- '`pie`' — круговая диаграмма;  
- '`scatter`' — диаграмма рассеяния.  

`grid` — отображение сетки (по умолчанию False);  
`legend` — отображение легенды (по умолчанию False);  
`title` — название графика;  
`color` — цвет.

Давайте попрактикуемся в использовании метода `plot()`.

Начнём с исследования заболеваемости коронавирусом во всём мире. Для этого первым делом отобразим, как менялось ежедневное число заболевших (`daily_confirmed`) во всём мире во времени. Далее сгруппируем таблицу по датам и подсчитаем суммарное число зафиксированных случаев по дням.

Теперь мы наконец можем построить график с помощью метода `plot()`. Будем использовать линейный график размером 12x4 (попробуйте взять другие числа, чтобы увидеть разницу). Подпишем график и отобразим сетку. Параметр `lw` (`line width`) отвечает за ширину линии для линейного графика.